In [1]:
import warnings 
warnings.simplefilter('ignore')

import scanpy as sc
import muon as mu
import scparadise
import pandas as pd
import os

In [2]:
# Load normalized integrated data
mdata = mu.read_h5mu('Heart_3p_CITE/mdata_unintegrated_annotated.h5mu')

In [3]:
# Subset anndata object based on a selected marker genes
genes = pd.read_csv('Heart_3p_CITE/genes_for_AI.csv')
mdata.mod['rna'] = mdata.mod['rna'][:, genes.genes].copy()
mdata.update()
mdata = mdata[mdata.obs['celltype_l2'] != 'classical DC1']
mdata = mdata[mdata.obs['celltype_l2'] != 'Neutrophil']
mdata = mdata[mdata.obs['celltype_l2'] != 'plasmacytoid DC']
mdata = mdata[mdata.obs['celltype_l2'] != 'ILC']
mdata = mdata[mdata.obs['celltype_l2'] != 'proliferating T']

In [4]:
# Create list of samples to leave in reference train dataset
lst_reference = [12, 13, 17, 27, 28, 29, 30, 32, 39, 42]

In [5]:
# Create mdata_train
mdata_train = mdata[mdata.obs['orig.ident'].isin(lst_reference)].copy()

In [6]:
# Train default scEve model using mdata_train dataset
scparadise.sceve.train(mdata_train, 
                       path = '',
                       rna_modality_name = 'rna',
                       second_modality_name = 'adt',
                       detailed_annotation = 'celltype_l2',
                       model_name = 'model_Heart_scEve_default',
                       max_epochs = 200,
                       eval_metric = ['rmse'])

Successfully saved genes names for training model

Successfully saved proteins names for training model

Train dataset contains: 57832 cells, it is 90.0 % of input dataset
Test dataset contains: 6426 cells, it is 10.0 % of input dataset

Accelerator: cuda
Start training
epoch 0  | loss: 0.41689 | train_rmse: 0.47679999470710754| valid_rmse: 0.47773998975753784|  0:00:04s
epoch 1  | loss: 0.16476 | train_rmse: 0.4636099934577942| valid_rmse: 0.46404001116752625|  0:00:07s
epoch 2  | loss: 0.15055 | train_rmse: 0.4341999888420105| valid_rmse: 0.43428999185562134|  0:00:11s
epoch 3  | loss: 0.13819 | train_rmse: 0.42858999967575073| valid_rmse: 0.42868998646736145|  0:00:15s
epoch 4  | loss: 0.13554 | train_rmse: 0.424919992685318| valid_rmse: 0.42469000816345215|  0:00:18s
epoch 5  | loss: 0.13478 | train_rmse: 0.4239400029182434| valid_rmse: 0.42375999689102173|  0:00:22s
epoch 6  | loss: 0.13347 | train_rmse: 0.42236000299453735| valid_rmse: 0.4221400022506714|  0:00:26s
epoch 7  | los

In [7]:
# Create lists with test samples
lst_test_1 = ['1_6', '2_7', '4_5', '8_9']
lst_test_2 = ['41', '34', '15', '33']

In [8]:
for folder in lst_test_1:
    # Create adata_adt_test and adata_rna_test
    adata_adt_test = mdata.mod['adt'][mdata.mod['adt'].obs['orig.ident'].isin([int(folder[0]), int(folder[2])])].copy()
    adata_rna_test = mdata.mod['rna'][mdata.mod['rna'].obs['orig.ident'].isin([int(folder[0]), int(folder[2])])].copy()
    # Predict surface proteins using pretrained scEve model
    adata_pred_adt_test = scparadise.sceve.predict(adata_rna_test, 
                                                   path_model = 'model_Heart_scEve_default',
                                                   return_mdata = False)
    # Create and save regression report
    scparadise.scnoah.report_reg(adata_prot = adata_adt_test, 
                                 adata_pred_prot = adata_pred_adt_test, 
                                 report_name = folder + '_report_regression.csv', 
                                 save_path = 'Heart_3p_CITE/reports_model_Heart_scEve_default', 
                                 save_report = True)
    df_corr = pd.DataFrame(columns=['Pearson coef', 'p-value'])
    for i in adata_adt_test.var_names.tolist():
        person_coef = scparadise.scnoah.pearson_coef_prot(adata_prot = adata_adt_test,
                                                          adata_pred_prot = adata_pred_adt_test,
                                                          protein = i,
                                                          protein_pred = i + '_pred')
        df_corr.loc[i] = [person_coef['Pearson coefficient'], person_coef['p-value']]
    df_corr['p-value'] = df_corr['p-value'].astype('float64')
    file = "results_pearson_" + folder + ".csv"
    df_corr.to_csv(os.path.join('Heart_3p_CITE/reports_model_Heart_scEve_default', file))
for folder in lst_test_2:
    # Create adata_adt_test and adata_rna_test 
    adata_adt_test = mdata.mod['adt'][mdata.mod['adt'].obs['orig.ident'].isin([int(folder)])].copy()
    adata_rna_test = mdata.mod['rna'][mdata.mod['rna'].obs['orig.ident'].isin([int(folder)])].copy()
    # Predict surface proteins using pretrained scEve model
    adata_pred_adt_test = scparadise.sceve.predict(adata_rna_test, 
                                                   path_model = 'model_Heart_scEve_default',
                                                   return_mdata = False)
    # Create and save regression report
    scparadise.scnoah.report_reg(adata_prot = adata_adt_test, 
                                 adata_pred_prot = adata_pred_adt_test, 
                                 report_name = folder + '_report_regression.csv', 
                                 save_path = 'Heart_3p_CITE/reports_model_Heart_scEve_default', 
                                 save_report = True)
    df_corr = pd.DataFrame(columns=['Pearson coef', 'p-value'])
    for i in adata_adt_test.var_names.tolist():
        person_coef = scparadise.scnoah.pearson_coef_prot(adata_prot = adata_adt_test,
                                                          adata_pred_prot = adata_pred_adt_test,
                                                          protein = i,
                                                          protein_pred = i + '_pred')
        df_corr.loc[i] = [person_coef['Pearson coefficient'], person_coef['p-value']]
    df_corr['p-value'] = df_corr['p-value'].astype('float64')
    file = "results_pearson_" + folder + ".csv"
    df_corr.to_csv(os.path.join('Heart_3p_CITE/reports_model_Heart_scEve_default', file))

Successfully loaded list of genes used for training model

Successfully loaded list of features used for training model

Successfully loaded model

Successfully saved report

Successfully loaded list of genes used for training model

Successfully loaded list of features used for training model

Successfully loaded model

Successfully saved report

Successfully loaded list of genes used for training model

Successfully loaded list of features used for training model

Successfully loaded model

Successfully saved report

Successfully loaded list of genes used for training model

Successfully loaded list of features used for training model

Successfully loaded model

Successfully saved report

Successfully loaded list of genes used for training model

Successfully loaded list of features used for training model

Successfully loaded model

Successfully saved report

Successfully loaded list of genes used for training model

Successfully loaded list of features used for training model

Succ

In [9]:
scparadise.sceve.hyperparameter_tuning(mdata_train, 
                                       path = '',
                                       rna_modality_name = 'rna',
                                       second_modality_name = 'adt',
                                       detailed_annotation = 'celltype_l2',
                                       random_state = 42,
                                       model_name = 'model_Heart_scEve_tuning',
                                       eval_metric = ['rmse'])

Successfully saved genes names for training model

Successfully saved proteins names for training model

Accelerator: cuda



[I 2025-01-23 11:17:39,732] A new study created in RDB with name: model_Heart_scEve_tuning


Fold 1:

Early stopping occurred at epoch 90 with best_epoch = 80 and best_valid_rmse = 0.40928998589515686

Fold 2:
Stop training because you reached max_epochs = 100 with best_epoch = 97 and best_valid_rmse = 0.4117499887943268

Fold 3:
Stop training because you reached max_epochs = 100 with best_epoch = 90 and best_valid_rmse = 0.4060800075531006

Fold 4:
Stop training because you reached max_epochs = 100 with best_epoch = 95 and best_valid_rmse = 0.4094499945640564


[I 2025-01-23 11:35:11,352] Trial 0 finished with value: 0.40914224833250046 and parameters: {'n_d': 8, 'n_a': 8, 'n_steps': 3, 'n_shared': 2, 'cat_emb_dim': 1, 'n_independent': 1, 'gamma': 1.3, 'momentum': 0.02, 'lr': 0.01, 'mask_type': 'entmax', 'lambda_sparse': 0.001, 'patience': 10, 'max_epochs': 100, 'virtual_batch_size': 128, 'batch_size': 1024}. Best is trial 0 with value: 0.40914224833250046.



Fold 1:
Stop training because you reached max_epochs = 45 with best_epoch = 34 and best_valid_rmse = 0.47350001335144043

Fold 2:

Early stopping occurred at epoch 32 with best_epoch = 12 and best_valid_rmse = 0.45684999227523804

Fold 3:

Early stopping occurred at epoch 36 with best_epoch = 16 and best_valid_rmse = 0.45882999897003174

Fold 4:

Early stopping occurred at epoch 37 with best_epoch = 17 and best_valid_rmse = 0.44534000754356384


...

[I 2025-01-24 01:37:01,165] Trial 98 finished with value: 0.3978909105062485 and parameters: {'n_d': 124, 'n_a': 116, 'n_steps': 3, 'n_shared': 2, 'cat_emb_dim': 8, 'n_independent': 4, 'gamma': 1.915142248878431, 'momentum': 0.05545881020033835, 'lr': 0.025398043433805306, 'mask_type': 'sparsemax', 'lambda_sparse': 0.00038751154915366183, 'patience': 15, 'max_epochs': 160, 'virtual_batch_size': 128, 'batch_size': 512}. Best is trial 84 with value: 0.3958360478281975.



Fold 1:

Early stopping occurred at epoch 47 with best_epoch = 32 and best_valid_rmse = 0.3999199867248535

Fold 2:

Early stopping occurred at epoch 44 with best_epoch = 29 and best_valid_rmse = 0.40116000175476074

Fold 3:

Early stopping occurred at epoch 58 with best_epoch = 43 and best_valid_rmse = 0.39715999364852905


[I 2025-01-24 01:48:55,511] Trial 99 pruned. 



Successfully saved best hyperparameters

Best hyperparameters: {'n_d': 124, 'n_a': 100, 'n_steps': 4, 'n_shared': 3, 'cat_emb_dim': 9, 'n_independent': 5, 'gamma': 1.8487718599345104, 'momentum': 0.05372740601621637, 'mask_type': 'entmax', 'lambda_sparse': 0.0002534570741356221, 'patience': 15, 'max_epochs': 165, 'virtual_batch_size': 128, 'batch_size': 640, 'optimizer_params': {'lr': 0.017155146443617965}}


{'n_d': 124,
 'n_a': 100,
 'n_steps': 4,
 'n_shared': 3,
 'cat_emb_dim': 9,
 'n_independent': 5,
 'gamma': 1.8487718599345104,
 'momentum': 0.05372740601621637,
 'mask_type': 'entmax',
 'lambda_sparse': 0.0002534570741356221,
 'patience': 15,
 'max_epochs': 165,
 'virtual_batch_size': 128,
 'batch_size': 640,
 'optimizer_params': {'lr': 0.017155146443617965}}

In [10]:
# Train sceve model using mdata_train dataset
scparadise.sceve.train_tuned(mdata_train, 
                             path = '',
                             path_tuned='model_Heart_scEve_tuning',
                             rna_modality_name = 'rna',
                             second_modality_name = 'adt',
                             detailed_annotation = 'celltype_l2',
                             model_name = 'model_Heart_scEve_tuned',
                             eval_metric=['rmse'])

Successfully saved genes names for training model

Successfully saved proteins names for training model

Train dataset contains: 57832 cells, it is 90.0 % of input dataset
Test dataset contains: 6426 cells, it is 10.0 % of input dataset

Accelerator: cuda
Start training with following hyperparameters: {'n_d': 124, 'n_a': 100, 'n_steps': 4, 'n_shared': 3, 'cat_emb_dim': 9, 'n_independent': 5, 'gamma': 1.8487718599345104, 'momentum': 0.05372740601621637, 'mask_type': 'entmax', 'lambda_sparse': 0.0002534570741356221, 'patience': 15, 'max_epochs': 165, 'virtual_batch_size': 128, 'batch_size': 640, 'optimizer_params': {'lr': 0.017155146443617965}, 'device_name': 'cuda'}

epoch 0  | loss: 1.02587 | train_rmse: 0.4382700026035309| valid_rmse: 0.4378899931907654|  0:00:06s
epoch 1  | loss: 0.13475 | train_rmse: 0.4249599874019623| valid_rmse: 0.4248400032520294|  0:00:12s
epoch 2  | loss: 0.13098 | train_rmse: 0.41773998737335205| valid_rmse: 0.4177500009536743|  0:00:19s
epoch 3  | loss: 0.12

In [11]:
for folder in lst_test_1:
    # Create adata_adt_test and adata_rna_test
    adata_adt_test = mdata.mod['adt'][mdata.mod['adt'].obs['orig.ident'].isin([int(folder[0]), int(folder[2])])].copy()
    adata_rna_test = mdata.mod['rna'][mdata.mod['rna'].obs['orig.ident'].isin([int(folder[0]), int(folder[2])])].copy()
    # Predict surface proteins using pretrained scEve model
    adata_pred_adt_test = scparadise.sceve.predict(adata_rna_test, 
                                                   path_model = 'model_Heart_scEve_tuned',
                                                   return_mdata = False)
    # Create and save regression report
    scparadise.scnoah.report_reg(adata_prot = adata_adt_test, 
                                 adata_pred_prot = adata_pred_adt_test, 
                                 report_name = folder + '_report_regression.csv', 
                                 save_path = 'Heart_3p_CITE/reports_model_Heart_scEve_tuned', 
                                 save_report = True)
    df_corr = pd.DataFrame(columns=['Pearson coef', 'p-value'])
    for i in adata_adt_test.var_names.tolist():
        person_coef = scparadise.scnoah.pearson_coef_prot(adata_prot = adata_adt_test,
                                                          adata_pred_prot = adata_pred_adt_test,
                                                          protein = i,
                                                          protein_pred = i + '_pred')
        df_corr.loc[i] = [person_coef['Pearson coefficient'], person_coef['p-value']]
    df_corr['p-value'] = df_corr['p-value'].astype('float64')
    file = "results_pearson_" + folder + ".csv"
    df_corr.to_csv(os.path.join('Heart_3p_CITE/reports_model_Heart_scEve_tuned', file))
for folder in lst_test_2:
    # Create adata_adt_test and adata_rna_test 
    adata_adt_test = mdata.mod['adt'][mdata.mod['adt'].obs['orig.ident'].isin([int(folder)])].copy()
    adata_rna_test = mdata.mod['rna'][mdata.mod['rna'].obs['orig.ident'].isin([int(folder)])].copy()
    # Predict surface proteins using pretrained scEve model
    adata_pred_adt_test = scparadise.sceve.predict(adata_rna_test, 
                                                   path_model = 'model_Heart_scEve_tuned',
                                                   return_mdata = False)
    # Create and save regression report
    scparadise.scnoah.report_reg(adata_prot = adata_adt_test, 
                                 adata_pred_prot = adata_pred_adt_test, 
                                 report_name = folder + '_report_regression.csv', 
                                 save_path = 'Heart_3p_CITE/reports_model_Heart_scEve_tuned', 
                                 save_report = True)
    df_corr = pd.DataFrame(columns=['Pearson coef', 'p-value'])
    for i in adata_adt_test.var_names.tolist():
        person_coef = scparadise.scnoah.pearson_coef_prot(adata_prot = adata_adt_test,
                                                          adata_pred_prot = adata_pred_adt_test,
                                                          protein = i,
                                                          protein_pred = i + '_pred')
        df_corr.loc[i] = [person_coef['Pearson coefficient'], person_coef['p-value']]
    df_corr['p-value'] = df_corr['p-value'].astype('float64')
    file = "results_pearson_" + folder + ".csv"
    df_corr.to_csv(os.path.join('Heart_3p_CITE/reports_model_Heart_scEve_tuned', file))

Successfully loaded list of genes used for training model

Successfully loaded list of features used for training model

Successfully loaded model

Successfully saved report

Successfully loaded list of genes used for training model

Successfully loaded list of features used for training model

Successfully loaded model

Successfully saved report

Successfully loaded list of genes used for training model

Successfully loaded list of features used for training model

Successfully loaded model

Successfully saved report

Successfully loaded list of genes used for training model

Successfully loaded list of features used for training model

Successfully loaded model

Successfully saved report

Successfully loaded list of genes used for training model

Successfully loaded list of features used for training model

Successfully loaded model

Successfully saved report

Successfully loaded list of genes used for training model

Successfully loaded list of features used for training model

Succ

In [12]:
import session_info
session_info.show()